In [1]:
## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']


In [2]:
tss_annotation = doc['tss_annotation']

In [3]:

import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 13)


In [4]:
genes_to_look = ['MGAT3','ID1']

In [5]:
bed_files = []
signal_files = []

In [6]:
serv = '/mnt/jabba'

out_dir = "/data/isshamie/TSS/Analysis/Analysis_NCBI_PICR"
out_dir_save = os.path.join(out_dir,"Results/Figures/Figure1")
if not os.path.exists(out_dir_save):
    os.mkdir(out_dir_save)

In [7]:
tss = pd.read_csv("Results/output/TSS1.bed",sep="\t",header=None)
tss.rename({0:"Chr",1:"Start",2:"End",3:"ID",4:"Stat",5:"Strand"},axis=1,inplace=True)
tss.head()

,Chr,Start,End,ID,Stat,Strand
0,NW_020822366.1,721599,721750,p2@Bmp4_XM_027386529.1,0.506505,-
1,NW_020822366.1,721832,721983,p3@Bmp4_XM_027386529.1,0.580925,-
2,NW_020822366.1,722208,722359,p1@Bmp4_XM_027386529.1,1.779596,-
3,NW_020822366.1,724533,724684,p1@Bmp4_XM_027386528.1,4.074290,-
4,NW_020822366.1,1085431,1085582,p1@Cdkn3_XM_027386531.1,1.212188,+


In [8]:
vary_width = [0,100,1000,10000]

## For Publication:
-ATAC
-GROCap
-GRO
-START
-STARTinput
-TSS
-RefSeq
-dTSS

In [9]:
curr_folder = '/data/isshamie/TSS/Analysis/Analysis_NCBI_PICR/'
igv_script = """new
genome {serv}{ref_fa}
load {serv}{annotation}

load {serv}{out_dir}/Results/output/TSS1.bed
load {serv}{out_dir}/Results/merged/tags_TSS_merged.bedgraph.gz

# CHO
load {serv}{data_folder}/CHO/ATAC/out/signal/macs2/pooled_rep/CHO_ATAC1.trim.nodup.tn5_CHO_ATAC3.trim.nodup.tn5.pf.pval.signal.bigwig
load {serv}{data_folder}/CHO/GROCap/bedgraph_files/trim_CHO_GRO1.bedgraph.gz 
load {serv}{data_folder}/CHO/GROCap/bedgraph_files/trim_CHO_GROCap1.bedgraph.gz 

load {serv}{data_folder}/CHO/START/bedgraph_files/trim_CHO_START1.bedgraph.gz 
load {serv}{data_folder}/CHO/START/bedgraph_files/trim_CHO_STARTinput1.bedgraph.gz 

# Brain
load {serv}{data_folder}/Brain/ATAC/out/signal/macs2/rep1/Brain_ATAC1.trim.nodup.tn5.pf.pval.signal.bigwig
load {serv}{data_folder}/Brain/GRO/bedgraph_files/trim_Brain_GROCap1.bedgraph.gz 
load {serv}{data_folder}/Brain/GRO/bedgraph_files/trim_Brain_GRO1.bedgraph.gz 

#Liver 
load {serv}{data_folder}/Liver/GRO/bedgraph_files/trim_Liver_GROCap1.bedgraph.gz
load {serv}{data_folder}/Liver/ATAC/out/signal/macs2/rep1/Liver_ATAC1.trim.nodup.tn5.pf.pval.signal.bigwig

snapshotDirectory {serv}{out_dir}/Results/Figures/Figure1""".format(
    serv=serv,
    data_folder=data_folder,
    out_dir=out_dir,
    ref_fa=ref_fa,
    annotation=annotation)

snaps = "snapshotDirectory {serv}{out_dir}/Results/Figures/Figure1/\n".format(
    serv=serv, out_dir=out_dir)

for ind, val in tss[tss["ID"].str.upper().str.contains("MGAT3")].iterrows():
    for w in vary_width:
        snaps = snaps + 'goto {chrom}:{start}-{end}\nsort\nsnapshot {name}_{w}.png\nsnapshot {name}_{w}.svg\n'.format(
            chrom=tss.loc[ind, "Chr"],
            start=tss.loc[ind, "Start"] - w,
            end=tss.loc[ind, "End"] + w,
            name=tss.loc[ind, "ID"],
            w=w)

# for i in glob.glob("f04_peaks_bed/*bed"):
#     igv_script = igv_script +  "\nload " + serv + os.path.join(curr_folder, i)
if not os.path.exists(os.path.join(out_dir_save,"bed_files/")):
    os.mkdir(os.path.join(out_dir_save,"bed_files/"))
    
for t in tissues:
    for s in ["GROCap", "START"]:
        for i in glob.glob(
                os.path.join(data_folder, t, s, "bed_files/", "*merge*bed")):
            name = os.path.join(out_dir_save,"bed_files/" + t + "_" + s + ".bed")
            if not os.path.exists(name):
                cmd = "cp {i} {name}".format(i=i,name=name)
                !{cmd}
            #igv_script = os.path.join(s)
            igv_script = igv_script + "\nload " + serv + name

with open('Results/Figures/Figure1/igv_batch_TSS_Pub_LOAD.bat', 'w') as f:
    f.write(igv_script)

batch = igv_script + "\n" + snaps

with open('Results/Figures/Figure1/igv_batch_TSS_MGAT3.bat', 'w') as f:
    f.write(batch)

with open('Results/Figures/Figure1/igv_batch_TSS_MGAT3_SNAP.bat', 'w') as f:
    f.write(snaps)

batch

'new\ngenome /mnt/jabba/data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna\nload /mnt/jabba/data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gff3\n\nload /mnt/jabba/data/isshamie/TSS/Analysis/Analysis_NCBI_PICR/Results/output/TSS1.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/Analysis_NCBI_PICR/Results/merged/tags_TSS_merged.bedgraph.gz\n\n# CHO\nload /mnt/jabba/data/isshamie/TSS/NCBI_PICR_103_Processed//CHO/ATAC/out/signal/macs2/pooled_rep/CHO_ATAC1.trim.nodup.tn5_CHO_ATAC3.trim.nodup.tn5.pf.pval.signal.bigwig\nload /mnt/jabba/data/isshamie/TSS/NCBI_PICR_103_Processed//CHO/GROCap/bedgraph_files/trim_CHO_GRO1.bedgraph.gz \nload /mnt/jabba/data/isshamie/TSS/NCBI_PICR_103_Processed//CHO/GROCap/bedgraph_files/trim_CHO_GROCap1.bedgraph.gz \n\nload /mnt/jabba/data/isshamie/TSS/NCBI_PICR_103_Processed//CHO/START/bedgraph_files/trim_CHO_START1.bedgraph.gz \nload /mnt/jabba/data/isshamie/TSS/NCBI_PICR_103_Processed//CHO/START/bedgraph_files/trim_CHO_STARTi

## 3 Genes of interest:
Mgat3  
ST6Gal1  
Aldh18a1  


In [16]:
snaps = "snapshotDirectory {serv}{out_dir}/Results/Figures/Figure1/\n".format(
    serv=serv, out_dir=out_dir)
GOI = ["MGAT3","ST6GAL1","ALDH18A1","MGAT5"]
for g in GOI:
    for ind, val in tss[tss["ID"].str.upper().str.contains(g)].iterrows():
        for w in vary_width:
            snaps = snaps + 'goto {chrom}:{start}-{end}\nsort\nsnapshot {name}_{w}.png\nsnapshot {name}_{w}.svg\n'.format(
                chrom=tss.loc[ind, "Chr"],
                start=tss.loc[ind, "Start"] - w,
                end=tss.loc[ind, "End"] + w,
                name=tss.loc[ind, "ID"],
                w=w)
            
with open('Results/Figures/Figure1/igv_batch_TSS_GOI_SNAP.bat', 'w') as f:
    f.write(snaps)


In [17]:
snaps = "snapshotDirectory {serv}{out_dir}/Results/Figures/Figure1/\n".format(
    serv=serv, out_dir=out_dir)
GOI = ["ALDH18A1"]
for g in GOI:
    for ind, val in tss[tss["ID"].str.upper().str.contains(g)].iterrows():
        for w in vary_width:
            snaps = snaps + 'goto {chrom}:{start}-{end}\nsort\nsnapshot {name}_{w}.png\nsnapshot {name}_{w}.svg\n'.format(
                chrom=tss.loc[ind, "Chr"],
                start=tss.loc[ind, "Start"] - w,
                end=tss.loc[ind, "End"] + w,
                name=tss.loc[ind, "ID"],
                w=w)
            
with open('Results/Figures/Figure1/igv_batch_TSS_ALDH18A1_SNAP.bat', 'w') as f:
    f.write(snaps)


### GOIs:

In [11]:
tss_meta = pd.read_csv("Results/output/TSS1.meta",sep="\t",index_col=0)
#tss_meta[tss_meta["Gene"].str.upper().str.contains("MGAT3")]


In [12]:
print(tss_meta[tss_meta["Gene"].str.upper().str.contains("MGAT3")]["Tissues"][0])
tss[tss["ID"].str.upper().str.contains("MGAT3")]


FemaleReproductive


,Chr,Start,End,ID,Stat,Strand
17534,NW_020822470.1,7502049,7502200,p1@Mgat3_NM_001244074.1,0.394452,+


In [13]:
print(tss_meta[tss_meta["Gene"].str.upper().str.contains("ALDH18A1")]["Tissues"][0])
#tss_meta[tss_meta["Gene"].str.upper().str.contains("ALDH18A1")]
tss[tss["ID"].str.upper().str.contains("ALDH18A1")]


Heart,Spleen,MiscOrgans,FemaleReproductive,BMDM1hKLA,Brain,Lung,Liver,BMDMwt,Muscle,Kidney,Pancreas


,Chr,Start,End,ID,Stat,Strand
20637,NW_020822501.1,20998652,20998803,p1@Aldh18a1_XM_027407359.1,2.587823,+


In [14]:
print(tss_meta[tss_meta["Gene"].str.upper().str.contains("ST6GAL1")]["Tissues"][0])
tss[tss["ID"].str.upper().str.contains("ST6GAL1")]

Liver


,Chr,Start,End,ID,Stat,Strand
26277,NW_020822529.1,57376916,57377067,p1@St6gal1_NM_001246815.1,0.746634,-


In [15]:
print(tss_meta[tss_meta["Gene"].str.upper().str.contains("MGAT5")]["Tissues"][0])
tss[tss["ID"].str.upper().str.contains("MGAT5")]


nan


,Chr,Start,End,ID,Stat,Strand
30798,NW_020822567.1,39762223,39762225,p0@Mgat5_XM_027417771.1,0.000000,-
30799,NW_020822567.1,39762362,39762513,p1@Mgat5_XM_027417770.1,0.769377,-
30800,NW_020822567.1,39813270,39813421,p1@Mgat5_XM_027417769.1,1.723456,-
38375,NW_020822611.1,4879714,4879865,p1@Mgat5b_XM_027425561.1,0.678518,-


In [35]:
tss_meta.head()

,Tissues,cs,CHO ATAC Region,ID,Gene,Gene ID,Transcript,Is Experimental,has ATAC
p1@Bmp4_XM_027386528.1,"Heart,Spleen,MiscOrgans,FemaleReproductive,Bra...",0b11,NaN,308710.0,Bmp4,NaN,XM_027386528.1,1,1
p1@Cdkn3_XM_027386531.1,"BMDMwt,Spleen,MiscOrgans,FemaleReproductive,CH...",0b11,NaN,237253.0,Cdkn3,NaN,XM_027386531.1,1,0
p0@Cdkn3_XM_027386533.1,NaN,NaN,NaN,NaN,Cdkn3,NaN,XM_027386533.1,0,0
p2@Cnih1_XM_027386534.1,"Heart,Spleen,MiscOrgans,FemaleReproductive,CHO...",0b11,1113861:1114585,353645.0,Cnih1,NaN,XM_027386534.1,1,1
p1@Cnih1_XM_027386534.1,"Heart,Spleen,MiscOrgans,FemaleReproductive,CHO...",0b11,NaN,298307.0,Cnih1,NaN,XM_027386534.1,1,0


In [36]:
tss_meta.loc["p1@Ktn1_XM_027386555.1"]

Tissues                 BMDM1hKLA
cs                           0b10
CHO ATAC Region               NaN
ID                          71494
Gene                         Ktn1
Gene ID                       NaN
Transcript         XM_027386555.1
Is Experimental                 1
has ATAC                        1
Name: p1@Ktn1_XM_027386555.1, dtype: object

##   
CHO, Brain ATAC  
CHO Start, Brain GRO/GROCap  
Start with MGAT3 and FemaleReproductive GRO/GROCap  
Then remove FemaleReproductive  
Add   

----------

----------

### Random gene TSS regions

In [17]:
snaps = 'snapshotDirectory {serv}{out_dir}/Results/Figures/Figure1\n'.format(serv=serv)
for i in range(len(tss)):
    if i % 2500 == 0:
        for w in vary_width:
            snaps = snaps + 'goto {chrom}:{start}-{end}\nsort\nsnapshot {name}_{w}.png\n'.format(
                chrom=tss.loc[i, "Chr"],
                start=tss.loc[i, "Start"] - w,
                end=tss.loc[i, "End"] + w,
                name=tss.loc[i, "ID"],
                w=w)
batch = igv_script + "\n" + snaps
with open('Results/Figures/Figure1/igv_batch_TSS.bat', 'w') as f:
    f.write(batch)

with open('Results/Figures/Figure1/igv_batch_TSS_SNAP.bat', 'w') as f:
    f.write(snaps)
    
batch



'new\ngenome /mnt/jabba/data/isshamie/genome/picr_final/picr.fa\nload /mnt/jabba/data/isshamie/genome/picr_final/final.gff3\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/Results/output/TSS1.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/Results/merged/tags_TSS_merged.bedgraph.gz\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/Results/merged/tags_input_merged.bedgraph.gz\nload /mnt/jabba/data/isshamie/TSS/1015_Processed//CHO/ATAC/out/signal/macs2/pooled_rep/CHO_ATAC1.trim.nodup.tn5_CHO_ATAC3.trim.nodup.tn5.pf.pval.signal.bigwig\n\nsnapshotDirectory /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/Results/igv_snapshots\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/f04_peaks_bed/FemaleReproductive_START.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/f04_peaks_bed/Spleen_START.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/f04_peaks_bed/Kidney_GROCap.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Resul

### Second: Load every single peak file and the final output and see the shift
Have a CHO ATAC and GRO as well

In [14]:
curr_folder = '{out_dir}'
igv_script = """new
genome {serv}{ref_fa}
load {serv}{annotation}
load {serv}{out_dir}/Results/output/TSS1.bed
load {serv}{out_dir}/Results/merged/tags_TSS_merged.bedgraph.gz
load {serv}{out_dir}/Results/merged/tags_input_merged.bedgraph.gz
load {serv}{data_folder}/CHO/ATAC/out/signal/macs2/pooled_rep/CHO_ATAC1.trim.nodup.tn5_CHO_ATAC3.trim.nodup.tn5.pf.pval.signal.bigwig

snapshotDirectory {serv}{out_dir}/Results/Figures/Figure1""".format(serv=serv,data_folder=data_folder)

for i in glob.glob("f04_peaks_bed/*bed"):
    igv_script = igv_script +  "\nload " + serv + os.path.join(curr_folder, i)
    
igv_script

'new\ngenome /mnt/jabba/data/isshamie/genome/picr_final/picr.fa\nload /mnt/jabba/data/isshamie/genome/picr_final/final.gff3\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/Results/output/TSS1.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/Results/merged/tags_TSS_merged.bedgraph.gz\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/Results/merged/tags_input_merged.bedgraph.gz\nload /mnt/jabba/data/isshamie/TSS/1015_Processed//CHO/ATAC/out/signal/macs2/pooled_rep/CHO_ATAC1.trim.nodup.tn5_CHO_ATAC3.trim.nodup.tn5.pf.pval.signal.bigwig\n\nsnapshotDirectory /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/Results/igv_snapshots\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/f04_peaks_bed/FemaleReproductive_START.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/f04_peaks_bed/Spleen_START.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/f04_peaks_bed/Kidney_GROCap.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Resul

### MGAT regions

In [16]:
snaps = 'snapshotDirectory {serv}{out_dir}/Results/Figures/Figure1\n'.format(serv=serv)


for ind,val in tss[tss["ID"].str.contains("MGAT")].iterrows():
    
    for w in vary_width:
        snaps = snaps + 'goto {chrom}:{start}-{end}\nsort\ncollapse\nsnapshot {name}_{w}\n'.format(
            chrom=tss.loc[ind, "Chr"],
            start=tss.loc[ind, "Start"] - w,
            end=tss.loc[ind, "End"] + w,
            name=tss.loc[ind, "ID"],
            w=w)

batch = igv_script + "\n" + snaps
with open('Results/Figures/Figure1/igv_batch_TSS_MGAT.bat', 'w') as f:
    f.write(batch)
batch


with open('Results/Figures/Figure1/igv_batch_TSS_MGAT_SNAP.bat', 'w') as f:
    f.write(snaps)